In [ ]:
import infomap as im
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [ ]:
%matplotlib notebook

In [ ]:
G = nx.read_gml("OmicsIntegratorRun/recAndRanklForestsColored.gml", label = "id")

Following [these examples](https://github.com/mapequation/infomap-notebooks/blob/master/1_1_infomap_intro.ipynb).

In [ ]:
def findCommunities(G):
    """
    Partition network with the Infomap algorithm.
    Annotates nodes with 'community' id and return number of communities found.
    """
    infomapX = im.Infomap("--markov-time 2.5 --two-level")

    print("Building Infomap network from a NetworkX graph...")
    for e in G.edges().data():
        infomapX.network().addLink(*(e[0], e[1], e[2]["weight"]))

    print("Find communities with Infomap...")
    infomapX.run();

    print("Found {} modules with codelength: {}".format(infomapX.numTopModules(), infomapX.codelength()))

    communities = {}
    for node in infomapX.iterLeafNodes():
        communities[node.physicalId] = node.moduleIndex()

    nx.set_node_attributes(G, values=communities, name='community')

    print("\nIterate over physical nodes to get the overlapping physical module structure:")
    print("#physicalId moduleIndex flow")
    for node in infomapX.iterTreePhysical():
        if node.isLeaf():
            print("{} {} {}".format(node.physicalId, node.moduleIndex(), node.data.flow))

    infomapX.writeMap("net.map")
    infomapX.writeFlowTree("net.ftree")

In [ ]:
def drawNetwork(G):
    # position map
    pos = nx.spring_layout(G)
    # community ids
    communities = [v for k,v in nx.get_node_attributes(G, 'community').items()]
    numCommunities = max(communities) + 1
    # color map from http://colorbrewer2.org/
    cmapLight = plt.cm.get_cmap('PiYG', numCommunities)
    cmapDark = colors.ListedColormap(['black', 'firebrick', 'royalblue'], 'indexed', numCommunities)

    # Draw edges
    nx.draw_networkx_edges(G, pos)

    # Draw nodes
    nodeCollection = nx.draw_networkx_nodes(G,
        pos = pos,
        node_color = communities,
        cmap = cmapLight
    )
    # Set node border color to the darker shade
    darkColors = [cmapDark(v) for v in communities]
    nodeCollection.set_edgecolor(darkColors)

    # Draw node labels
    for n in G.nodes():
        plt.annotate(n,
            xy = pos[n],
            textcoords = 'offset points',
            horizontalalignment = 'center',
            verticalalignment = 'center',
            xytext = [0, 0],
            color = cmapDark(communities[n])
        )

    plt.axis('off')
    plt.show(figsize=(18, 16))

In [ ]:
findCommunities(G)
drawNetwork(G)

In [ ]:
import IPython
print(IPython.sys_info())